In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install import_ipynb
import import_ipynb
from model_test_for_openmax_add_CDF import*
import model_test_for_openmax_add_CDF 

In [ ]:
input_data = input()         # 아직 입력부를 구현하지 않아 임시로 입력

In [ ]:
data_declaration = {
    'comment' : ["str_Declaration"],
    'label':[-1]
}


Idata = pd.DataFrame(data_declaration)
Idata['comment'] = input_data

In [ ]:
x_data = Idata['comment']
y_data = Idata['label']

In [ ]:
def tokenize(x_data, tokenizer):
    tokenizer.fit_on_texts(x_data) # 데이터의 각 행별로 토큰화 수행
    return tokenizer.texts_to_sequences(x_data)    

In [ ]:
tokenizer = Tokenizer()
sequences = tokenize(x_data, tokenizer)

In [ ]:
X_data = sequences
X_result = pad_sequences(X_data, maxlen = maxlen)

## 모델 적용

In [ ]:
!pip install keras.models
from keras.models import model_from_json
json_file = open("model.json", "r")
loaded_model_json = json_file.read() 
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
loaded_model.load_weights("model_weight.h5")

In [ ]:
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=["accuracy",f1,precision, recall])
es = EarlyStopping(monitor='loss', mode='min' , min_delta=0)
check_point = ModelCheckpoint('apply_model.h5', monitor='loss', mode='min', save_best_only=True)

In [ ]:
loaded_model.summary()

In [ ]:
X_result = loaded_model.predict(X_data)
X_result = np.array(X_result).flatten().tolist()
print(X_result)

## logit Vector, Distance 계산

In [ ]:
class_result = np.where( X_result == np.max(X_result))
#class_result = np.where(X_result == max(X_result))
#tf_result = (y_data == class_result) # y_data : y 결과값(클래스 분류 값)
for_open_max_0 = []
for_open_max_1 = []
for_open_max_2 = []
print(class_result)
if(class_result == 0):
    for_open_max_0 = X_result
if(class_result == 1):
    for_open_max_1 = X_result
if(class_result == 2):
    for_open_max_2 = X_result

In [ ]:
def distance(logit_vector, average):
    m = []
    for i in range(0,3):
        m.append(logit_vector[i] - average[i])
    distance = (m[0]**2) + (m[1]**2) + (m[2]**2)
    return distance

dist0 = distance(X_result, average_0)
dist1 = distance(X_result, average_1)
dist2 = distance(X_result, average_2)
print(dist0)
print(dist1)
print(dist2)

## CDF 계산

In [ ]:
import scipy.stats as s

def calculCDF(dist):
    CDF = s.exponweib.cdf(i, *s.exponweib.fit(i, 1, 1, scale=2, loc=0))   
    return CDF
CDF0 = calculCDF(dist0)
CDF1 = calculCDF(dist1)
CDF2 = calculCDF(dist2)

## Logit Vetcr 업데이트

In [ ]:
updated_logit = []
i = 0

logit0 = X_result[0]-(X_result[0]*CDF0)
logit1 = X_result[1]-(X_result[1]*CDF1)
logit2 = X_result[2]-(X_result[2]*CDF2)
unkn_logit = CDF0*X_result[0] + CDF1*X_result[1] + CDF2*X_result[2]    # unknown class의 logit vector
updated_logit =[logit0, logit1, logit2, unkn_logit]
print(updated_logit)

## Softmax Layer 통과

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
per = softmax(updated_logit)
final_class = np.where( per == max(per))
print(final_class)
print(per)

## 결과 확인

In [ ]:
#print("테스트 결과(accuracy, f1, precision, recall): %.4f % (loaded_model.evaluate(X_data, final_class)[1]))
score = loaded_model.evaluate(X_data, final_class)
print("테스트 결과 : %.4f", % score)

## 모델 저장

In [ ]:
##모델을 .json 파일 형식으로 save하여 저장
model_json = model.to_json()
with open("new_model.json", "w") as json_file : 
    json_file.write(model_json)
    
model.save_weights("new_model_weight.h5")
model.save('new_full_model.h5')